In [1]:
import torch
from chinesebert import ChineseBertForMaskedLM, ChineseBertTokenizerFast, ChineseBertConfig
pretrained_model_name = "/home/yinxj/myPM/ChineseBERT-large"
tokenizer = ChineseBertTokenizerFast.from_pretrained(pretrained_model_name)
chinese_bert = ChineseBertForMaskedLM.from_pretrained(pretrained_model_name)


/home/yinxj/anaconda3/envs/csctuning/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ChineseBertTokenizerFast'.


In [22]:
text = '我是一个好人'
inputs = tokenizer(text, return_tensors="pt")['input_ids'][0]

text = '我是一个坏人'
inputs1 = tokenizer(text, return_tensors="pt")['input_ids'][0]
for i, (x, y) in enumerate(zip(inputs, inputs1)):
	if x != y:
		print(i)

5


In [13]:
def get_inputstr(src, tgt):
    srclist = list(src.strip())
    tgtlist = list(tgt.strip())
    inputlist = [x for x in srclist]
    masklist = []
    for i in range(len(srclist)):
        if srclist[i] != tgtlist[i]:
            masklist.append(i+1)
            inputlist[i] = "[MASK]"
    return "".join(inputlist), masklist
def predict(inputstr, masklist):
    text = inputstr
    maskpos = masklist
    inputs = tokenizer(text, return_tensors="pt")
    # print(inputs)
    predicts = []
    for pos in maskpos:
        with torch.no_grad():
            o = chinese_bert(**inputs)
            value, index = o.logits.softmax(-1)[0, pos].topk(10)

        pred_tokens = tokenizer.convert_ids_to_tokens(index.tolist())
        pred_values = value.tolist()

        outputs = []
        for t, p in zip(pred_tokens, pred_values):
            outputs.append(f"{t}|{round(p,4)}")
        predicts.append(outputs[0][0])
    return predicts

def get_output(predicts, srcline, masklist):
    srclist = list(srcline.strip())
    for p, m in zip(predicts, masklist):
        srclist[m-1] = p
    return "".join(srclist)



中
值


/home/yinxj/anaconda3/envs/csctuning/lib/python3.7/site-packages/transformers/modeling_utils.py:813: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


In [24]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='/home/yinxj/myPM/chinese_roberta_L-12_H-768')
unmasker("[MASK]国的首都是[MASK]京。")


[[{'score': 0.44081199169158936,
   'token': 704,
   'token_str': '中',
   'sequence': '[CLS] 中 国 的 首 都 是 [MASK] 京 。 [SEP]'},
  {'score': 0.17192570865154266,
   'token': 5401,
   'token_str': '美',
   'sequence': '[CLS] 美 国 的 首 都 是 [MASK] 京 。 [SEP]'},
  {'score': 0.1246388703584671,
   'token': 5739,
   'token_str': '英',
   'sequence': '[CLS] 英 国 的 首 都 是 [MASK] 京 。 [SEP]'},
  {'score': 0.0989055186510086,
   'token': 3791,
   'token_str': '法',
   'sequence': '[CLS] 法 国 的 首 都 是 [MASK] 京 。 [SEP]'},
  {'score': 0.04275975376367569,
   'token': 7506,
   'token_str': '韩',
   'sequence': '[CLS] 韩 国 的 首 都 是 [MASK] 京 。 [SEP]'}],
 [{'score': 0.6758550405502319,
   'token': 1266,
   'token_str': '北',
   'sequence': '[CLS] [MASK] 国 的 首 都 是 北 京 。 [SEP]'},
  {'score': 0.22043481469154358,
   'token': 691,
   'token_str': '东',
   'sequence': '[CLS] [MASK] 国 的 首 都 是 东 京 。 [SEP]'},
  {'score': 0.08935867995023727,
   'token': 1298,
   'token_str': '南',
   'sequence': '[CLS] [MASK] 国 的 首 都 是 南 京 。 [SEP]

In [31]:
unmasker("[MASK]的首都是[MASK]京。")[0][0]['token_str']

'我'

In [26]:
len('''的他们，怎会落得如此开局？赛后有球迷想不通也是自然。老实说，前两节结束时，笔者也有点发蒙。虽说由孙铭徽等三位国手撑场面的广厦男篮不改强队本色，但人家板凳厚底其实较上赛季是有所削弱的，而“升级版”的上海男篮也不至于输得如此彻底，如此溃不成军吧！图说：“寂寞大神”还未找回曾经的“神采”。是休赛期歇大了，还是练少了？某个角度看，好像是这么回事——前两个赛季得分如探囊取物的外援邓蒙，上场13分钟仅得到1分，感觉像变了个人；弗雷戴特也是在比赛中期，才慢慢找回手感，上场22分钟得14分，最多算及格；整支球队投篮命中不到4成，罚篮命中率更是不到7成，一些过去罚球很稳定的老将，开场连罚连失。第一反应：球队整体不在状态。图说：邓蒙？他是谁？他在哪儿?但细细分析比赛过程，发现真正让球队惨败的罪魁祸首，或许是另外两个——防守上的完全“不成体系”和进攻端队友间“极度陌生”。防守端的问题，和球队几位防守悍将的缺失或许有一定关系：内线双塔张兆旭和新人区俊炫双双伤缺，让本就没有大外援，仅剩下几位进攻性内线的“大鲨鱼”在防守端捉襟见肘。球队明显没有找到弥补这一短板的手段，退防速度、补防意识实在无法匹配广厦这样的对手。广厦男篮在比赛中采用大量挡拆掩护直接攻筐，且这一套路一用再用，但上海男篮就是找不到办法防住对手。队友间的“极度陌生”，体现在配合上。全场令人发指的27次失误，说实话，换任何一个对手，这要能赢都是奇迹了。而广厦男篮进攻端的“法宝”之一，恰恰就是利用对手的失误，或是抢下篮板球的机会下快攻。27次失误约等于27次送分，让对手轻轻松松吃上了大餐。如此一来，大败也就非常正常了。陌生的不止队友和队友，更有教练对球员。新任主帅斯帕夏此前强调过自己的执教风格——防守和提速。而比赛中，这两点几乎完全没有体现出来。有队员伤缺带来的阵容问题，也有他对球员的不熟悉导致的临场指挥问题。“赛前布置了注重对手的反击和挡拆，对手赢了30分，不是说去责备球员，我作为主教练，肯定会第一个来，最后一个走的。”斯帅赛后说。斯帅手中可打的牌其实并不少，他布置的防守策略或许也对。但是在比赛什么阶段重用什么特点的球员，采取什么样的搭配，显然他还没有摸到路子。这也直接导致球员之间配合生疏，技术发挥不出来，以及关键时候顶不住。而对于两位外援的表现，斯帅点评道，“他们刚隔离出来，一共合练两三次，篮球是团队运动，不能简单评价一两个人。”此外，他还表示新秀状元区俊炫因崴脚才缺席比赛，复出时间未定。当然，这个短暂的休赛期上海男篮是人员调动最大的一支球队，磨合需要时间，伤病需要回复。而赛季第一个对手广厦的特点，恰恰是球队主要结构、打法都是上赛季的老模式，球员默契并且打法早已成型，且本场比赛还有进一步的优化。孙铭徽、胡金秋等国手更是越发成熟。这些当然都是大败的理由，但问题是时间不等人，虽然难度不小，但赛季一开门上海男篮必须抓紧时间形成有效的阵容搭配和适合自己的打法。本赛季CBA虽然赛季拉长，但已经有足够多的历史经验告诉上海男篮——开头挖的坑越大，后面要逆转局势的可能就越低。''')

1269

### line to 500

In [28]:
def line2less500(fp):
	f = open(fp)
	lines = f.readlines()
	f = open(fp, 'w')
	for line in lines:
		f.write(line.strip()[:509] + '\n')
	f.close()

In [29]:
line2less500('/home/yinxj/mycode/csc/data/annotations/test.source')
line2less500('/home/yinxj/mycode/csc/data/annotations/test.target')